In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import gradio as gr
import requests
from PIL import Image
from io import BytesIO
import sqlite3
import os
from torch.utils.data import Dataset
import pandas as pd
import os



# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from PIL import Image
from torch.utils.data import Dataset
import pandas as pd
import os

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

class CollectionsDataset(Dataset):
    def __init__(self,
                 csv_file,
                 root_dir,
                 num_classes,
                 transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.num_classes = num_classes

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.data.loc[idx, 'img_name'])
        image = Image.open(img_name).convert('RGB')
        label_tensor = torch.zeros(self.num_classes)
        label_tensor[self.data.loc[idx, 'score']-1] = 1

        if self.transform:
            image = self.transform(image)

        return {'image': image,
                'labels': label_tensor
                }


In [ ]:
import torch.nn as nn
import pretrainedmodels as pm

model = pm.__dict__["resnet101"](pretrained='imagenet')

model.avg_pool = nn.AdaptiveAvgPool2d(1)
model.last_linear = nn.Sequential(
    nn.BatchNorm1d(2048),
    nn.Dropout(p=0.25),
    nn.Linear(in_features=2048, out_features=2048),
    nn.ReLU(),
    nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=2048, out_features=10),
)
# model.load_state_dict(torch.load("model.bin"))

In [ ]:
def train_model(model,
                data_loader,
                dataset_size,
                optimizer,
                scheduler,
                num_epochs):
    criterion = nn.BCEWithLogitsLoss()
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        scheduler.step()
        model.train()

        running_loss = 0.0
        # Iterate over data.
        for bi, d in enumerate(data_loader):
            inputs = d["image"]
            labels = d["labels"]
            inputs = inputs.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.float)

            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / dataset_size
        print('Loss: {:.4f}'.format(epoch_loss))
    return model

In [ ]:
import torch
from torchvision import transforms

# define some re-usable stuff
IMAGE_SIZE = 512
BATCH_SIZE = 9
device = torch.device("cuda:0")
IMG_MEAN = model.mean
IMG_STD = model.std

# make some augmentations on training data
train_transform=transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(IMG_MEAN, IMG_STD)
])


# use the collections dataset class we created earlier
CSV_FILE_TRAIN = r"dataset\balanced\data.csv"
ROOT_DIR_TRAIN = r"dataset\balanced"
NUM_CLASSES = 10

train_dataset = CollectionsDataset(CSV_FILE_TRAIN,ROOT_DIR_TRAIN,NUM_CLASSES, train_transform)

# create the pytorch data loader
train_dataset_loader = torch.utils.data.DataLoader(train_dataset,
                                                   batch_size=BATCH_SIZE,
                                                   shuffle=True)
# push model to device
model_ft = model.to(device)

In [ ]:
import torch.optim as optim
from torch.optim import lr_scheduler

plist = [
        {'params': model_ft.layer4.parameters(), 'lr': 1e-5},
        {'params': model_ft.last_linear.parameters(), 'lr': 5e-3}
        ]
optimizer_ft = optim.Adam(plist, lr=0.001)
lr_sch = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)


# model_ft = train_model(model_ft,
#                        train_dataset_loader,
#                        len(train_dataset),
#                        optimizer_ft,
#                        lr_sch,
#                        num_epochs=5)
#
# torch.save(model_ft.state_dict(), "model.bin")

In [ ]:
TEST_BATCH_SIZE = 10
test_transform=transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(IMG_MEAN,IMG_STD)
])

model_ft.load_state_dict(torch.load("model.bin"))
model_ft = model_ft.to(device)

for param in model_ft.parameters():
    param.requires_grad = False

model_ft.eval()

In [ ]:
reznya = Image.open(r"random_photos\image.jpg").convert('RGB')
reznya = train_transform(reznya).to(device)


In [ ]:
torch.argmax(model_ft(reznya.unsqueeze(0)).sigmoid()).item()+1